<a href="https://colab.research.google.com/github/cheyoung211/2023-Spring-Maths-for-ML/blob/main/Linear_models_for_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Modules

In [1]:
import pandas as pd
import numpy as np

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression

In [66]:
from sklearn.preprocessing import PolynomialFeatures

##Import Datasets

In [7]:
URL = 'https://raw.githubusercontent.com/cheyoung211/2023-Spring-Maths-for-ML/main/auto-mpg.csv'
df = pd.read_csv(URL)

In [8]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [9]:
df.describe()

,mpg,cylinders,displacement,weight,acceleration,model year,origin
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,5140.000000,24.800000,82.000000,3.000000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


##Preprocessing

In [34]:
df = df.drop(['car name'],axis = 1) #car name은 고려하지 않기로

In [35]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


In [38]:
df['horsepower'].value_counts()

150    22
90     20
88     19
110    18
100    17
       ..
61      1
93      1
148     1
152     1
82      1
Name: horsepower, Length: 93, dtype: int64

In [39]:
df['horsepower'].unique() #'?'라는 값이 있음

array(['130', '165', '150', '140', '198', '220', '215', '225', '190',
       '170', '160', '95', '97', '85', '88', '46', '87', '90', '113',
       '200', '210', '193', None, '100', '105', '175', '153', '180',
       '110', '72', '86', '70', '76', '65', '69', '60', '80', '54', '208',
       '155', '112', '92', '145', '137', '158', '167', '94', '107', '230',
       '49', '75', '91', '122', '67', '83', '78', '52', '61', '93', '148',
       '129', '96', '71', '98', '115', '53', '81', '79', '120', '152',
       '102', '108', '68', '58', '149', '89', '63', '48', '66', '139',
       '103', '125', '133', '138', '135', '142', '77', '62', '132', '84',
       '64', '74', '116', '82'], dtype=object)

In [40]:
#'?' 값을 지우고 knn으로 채우고 싶음
df['horsepower'] = df['horsepower'].replace('?',None)
imputer = KNNImputer(n_neighbors = 5)
df_imputed = imputer.fit_transform(df.iloc[:,0:])

In [48]:
type(df_imputed)  #array가 됨

numpy.ndarray

In [49]:
df = pd.DataFrame(df_imputed)

In [50]:
df.head()  # column 이름이 사라짐

,0,1,2,3,4,5,6,7
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0


In [52]:
df.columns = ['mpg','cylinders','displacement','horsepower','weight','acceleration','model year','origin']

In [53]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0


In [54]:
df['horsepower'].unique()  #잘 채워짐 근데 소숫점이 보기 싫음

array([130. , 165. , 150. , 140. , 198. , 220. , 215. , 225. , 190. ,
       170. , 160. ,  95. ,  97. ,  85. ,  88. ,  46. ,  87. ,  90. ,
       113. , 200. , 210. , 193. ,  62. , 100. , 105. , 175. , 153. ,
       180. , 110. ,  72. ,  86. ,  70. ,  76. ,  65. ,  69. ,  60. ,
        80. ,  54. , 208. , 155. , 112. ,  92. , 145. , 137. , 158. ,
       167. ,  94. , 107. , 230. ,  49. ,  75. ,  91. , 122. , 107.6,
        67. ,  83. ,  78. ,  52. ,  61. ,  93. , 148. , 129. ,  96. ,
        71. ,  98. , 115. ,  53. ,  81. ,  79. , 120. , 152. , 102. ,
       108. ,  68. ,  58. , 149. ,  89. ,  63. ,  48. ,  66. , 139. ,
       103. , 125. , 133. , 138. , 135. , 142. ,  77. ,  64.6, 132. ,
       112.8,  84. ,  64. ,  74. , 116. ,  88.2,  82. ])

In [57]:
df['horsepower'].round(decimals = 0)
df['horsepower'].astype(int)

0      130
1      165
2      150
3      150
4      140
      ... 
393     86
394     52
395     84
396     79
397     82
Name: horsepower, Length: 398, dtype: int64

In [58]:
x_train, x_test,y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0])

In [59]:
x_train.head()

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
166,8.0,302.0,129.0,3169.0,12.0,75.0,1.0
299,4.0,141.0,71.0,3190.0,24.8,79.0,2.0
369,4.0,112.0,88.0,2395.0,18.0,82.0,1.0
297,5.0,183.0,77.0,3530.0,20.1,79.0,2.0
123,6.0,156.0,122.0,2807.0,13.5,73.0,3.0


In [60]:
y_train.head()

166    13.0
299    27.2
369    34.0
297    25.4
123    20.0
Name: mpg, dtype: float64

##Linear Regression

In [61]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression()

In [64]:
reg_score1 = reg.score(x_test,y_test)
print('Linear regression :',reg_score1) # 1차

Linear regression : 0.819321640192419
